In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [3]:
!pip install gensim
!pip3 install konlpy
!pip install --upgrade jtypes.jpype
!pip3 install JPype1-py3
!pip install xlrd
!pip install pydblite
!pip install psutil

    100% |████████████████████████████████| 15.9MB 93kB/s  eta 0:00:01
    100% |████████████████████████████████| 1.4MB 1.1MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 7.8MB/s eta 0:00:01
    100% |████████████████████████████████| 4.0MB 387kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 8.9MB/s eta 0:00:01
    100% |████████████████████████████████| 552kB 2.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /root/.cache/pip/wheels/ed/64/56/a922ace26f5d32090849ec8d89192b2b9ff0d280a5be0a4b7b
  Running setup.py bdist_wheel for bz2file ... done
  Stored in directory: /root/.cache/pip/wheels/31/9c/20/996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built smart-open bz2file
    100% |████████████████████████████████| 22.5MB 66kB/s  eta 0:00:01
    100% |████████████████████████████████| 409kB 2.3MB/s ta 0:00:01
    100% |████████████████████████████████| 491kB 1.9MB/s eta 0:00:01
  Running s

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable   
from torch import optim
import numpy as np
import sys

sys.path.append("./Preprocessor")
sys.path.append("./Classifier")
sys.path.append('./Preprocessor/soy')
from Classifier.classifierModule import *
from Preprocessor.preprocessorModule import *
from Preprocessor.format_module import *

In [5]:
learning_rate = 0.005
input_size = 100  # word2vec k size
batch_size = 500
n_epochs = 40

In [6]:
reviewDB = ReviewDB("./Preprocessor/pkl/train")
format_module.FormattedReview.setDB(reviewDB)
FRlist = load_object("./Preprocessor/pkl/save_formatted_review_train.pkl")[:100]
model = classifierModule(input_size, batch_size, FRlist, "./Classifier/models/spam_filter.mdl")

In [7]:
def spamFilterModule(input_path, validation_path = None, sensitivity = 0.5, run_mode = 'default'):
    
    if(validation_path == None):
        formatted_review_list = preprocessModule(input_path, reviewDB, "test")
        out = model(formatted_review_list, None,run_mode)        
        tg = get_targets(formatted_review_list, model)
        _, formatted, _ = model.encoder(formatted_review_list)
        print()
        print("############# print test accuracy and infer results for each ##############")
        print(100 * get_accuracy(out, tg, 0.5)/len(formatted_review_list))
        for i in range(500):
            acc = get_accuracy(out[i:i+1], tg[i:i+1], 0.5)
            if tg.data[i]:
                print("            ",out.data[i, 1], tg.data[i], acc)
            else:
                print(out.data[i, 1], tg.data[i], acc)
            print("no excel for validation.. abort training")
            return
        
    else:
        formatted_review_list_for_training = preprocessModule(input_path, reviewDB, "train")
        formatted_review_list_for_validating = preprocessModule(validation_path, reviewDB, "validation")
        train_net(formatted_review_list_for_training, formatted_review_list_for_validating, model, reviewDB, sensitivity, run_mode)
        

In [9]:
spamFilterModule("./Preprocessor/excel/trainset_spam_ham_1to1.xlsx", True, "./Preprocessor/excel/Commonreviews_snuproject_validation.xlsx")

processing 0 th unit...
processing 10000 th unit...
processing 20000 th unit...
adding review to DB...
=> Postag initiated
postagging [komoran] 0 ith unit...


KeyboardInterrupt: 

In [ ]:
def setModel(path):
    model = classifierModule(input_size, batch_size, FRlist, "./Preprocessor/models/spam_filter.mdl", True)
    
def set_batch_size(size):
    batch_size = size

def set_max_epoch(num):
    n_epochs = num
    
def set_reiewDB(path):
    reviewDB = ReviewDB(path)
    format_module.FormattedReview.setDB(reviewDB)